In [86]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV
from joblib import dump, load
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

0       False
1        True
2        True
3        True
4        True
        ...  
8066    False
8067    False
8068    False
8069    False
8070    False
Name: playDirection, Length: 8071, dtype: bool

In [87]:
data = pd.read_csv("training_data_with_player_direction.csv")

C:\Users\Tad\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (389) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [88]:
# Transform Positions
data_transformed = data
data_transformed['play_direction_numerical'] = 1
data_transformed.loc[data_transformed['playDirection'] == 'left', 'play_direction_numerical'] = -1
delta_cols = []
for var_type in ['x', 'y', 's', 'a', 'dir', 'dis', 'o']: # note that magnitude vars (like speed and acceleration) are omitted.
    for pos in ['LT', 'LG', 'C', 'RG', 'RT']:
        col_name = var_type + "_0_" + pos
        for time in range(1, 11): # exclude 0, include 1.0
            time = round(time * 0.1, 1) # working with decimals
            time_str = str(time)
            if time == 1:
                time_str = '1'
            col_name = var_type + "_" + str(time_str) + "_" + pos
            print(col_name)
            prev_time = round(time - 0.1, 1)
            prev_time_str = str(prev_time)
            if prev_time == 0:
                prev_time_str = '0' # not '0.0'
            prev_time_col_name = var_type + "_" + str(prev_time_str) + "_" + pos
            delta_col_name = 'delta_' + col_name
            delta_cols.append(delta_col_name)
            data_transformed[delta_col_name] = (data_transformed[col_name] - data_transformed[prev_time_col_name]) * data_transformed['play_direction_numerical']

data_transformed[['x_0.1_LT', 'x_0_LT', 'delta_x_0.1_LT', 'playDirection']] # quick check

x_0.1_LT
x_0.2_LT
x_0.3_LT
x_0.4_LT
x_0.5_LT
x_0.6_LT
x_0.7_LT
x_0.8_LT
x_0.9_LT
x_1_LT
x_0.1_LG
x_0.2_LG
x_0.3_LG
x_0.4_LG
x_0.5_LG
x_0.6_LG
x_0.7_LG
x_0.8_LG
x_0.9_LG
x_1_LG
x_0.1_C
x_0.2_C
x_0.3_C
x_0.4_C
x_0.5_C
x_0.6_C
x_0.7_C
x_0.8_C
x_0.9_C
x_1_C
x_0.1_RG
x_0.2_RG
x_0.3_RG
x_0.4_RG
x_0.5_RG
x_0.6_RG
x_0.7_RG
x_0.8_RG
x_0.9_RG
x_1_RG
x_0.1_RT
x_0.2_RT
x_0.3_RT
x_0.4_RT
x_0.5_RT
x_0.6_RT
x_0.7_RT
x_0.8_RT
x_0.9_RT
x_1_RT
y_0.1_LT
y_0.2_LT
y_0.3_LT
y_0.4_LT
y_0.5_LT
y_0.6_LT
y_0.7_LT
y_0.8_LT
y_0.9_LT
y_1_LT
y_0.1_LG
y_0.2_LG
y_0.3_LG
y_0.4_LG
y_0.5_LG
y_0.6_LG
y_0.7_LG
y_0.8_LG
y_0.9_LG
y_1_LG
y_0.1_C
y_0.2_C
y_0.3_C
y_0.4_C
y_0.5_C
y_0.6_C
y_0.7_C
y_0.8_C
y_0.9_C
y_1_C
y_0.1_RG
y_0.2_RG
y_0.3_RG
y_0.4_RG
y_0.5_RG
y_0.6_RG
y_0.7_RG
y_0.8_RG
y_0.9_RG
y_1_RG
y_0.1_RT
y_0.2_RT
y_0.3_RT
y_0.4_RT
y_0.5_RT
y_0.6_RT
y_0.7_RT
y_0.8_RT
y_0.9_RT
y_1_RT
s_0.1_LT
s_0.2_LT
s_0.3_LT
s_0.4_LT
s_0.5_LT
s_0.6_LT
s_0.7_LT
s_0.8_LT
s_0.9_LT
s_1_LT
s_0.1_LG
s_0.2_LG
s_0.3_LG
s_0.4_LG
s_0.5_LG
s_0.6_L

<ipython-input-88-6ac72a48b748>:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data_transformed[delta_col_name] = (data_transformed[col_name] - data_transformed[prev_time_col_name]) * data_transformed['play_direction_numerical']


s_0.1_C
s_0.2_C
s_0.3_C
s_0.4_C
s_0.5_C
s_0.6_C
s_0.7_C
s_0.8_C
s_0.9_C
s_1_C
s_0.1_RG
s_0.2_RG
s_0.3_RG
s_0.4_RG
s_0.5_RG
s_0.6_RG
s_0.7_RG
s_0.8_RG
s_0.9_RG
s_1_RG
s_0.1_RT
s_0.2_RT
s_0.3_RT
s_0.4_RT
s_0.5_RT
s_0.6_RT
s_0.7_RT
s_0.8_RT
s_0.9_RT
s_1_RT
a_0.1_LT
a_0.2_LT
a_0.3_LT
a_0.4_LT
a_0.5_LT
a_0.6_LT
a_0.7_LT
a_0.8_LT
a_0.9_LT
a_1_LT
a_0.1_LG
a_0.2_LG
a_0.3_LG
a_0.4_LG
a_0.5_LG
a_0.6_LG
a_0.7_LG
a_0.8_LG
a_0.9_LG
a_1_LG
a_0.1_C
a_0.2_C
a_0.3_C
a_0.4_C
a_0.5_C
a_0.6_C
a_0.7_C
a_0.8_C
a_0.9_C
a_1_C
a_0.1_RG
a_0.2_RG
a_0.3_RG
a_0.4_RG
a_0.5_RG
a_0.6_RG
a_0.7_RG
a_0.8_RG
a_0.9_RG
a_1_RG
a_0.1_RT
a_0.2_RT
a_0.3_RT
a_0.4_RT
a_0.5_RT
a_0.6_RT
a_0.7_RT
a_0.8_RT
a_0.9_RT
a_1_RT
dir_0.1_LT
dir_0.2_LT
dir_0.3_LT
dir_0.4_LT
dir_0.5_LT
dir_0.6_LT
dir_0.7_LT
dir_0.8_LT
dir_0.9_LT
dir_1_LT
dir_0.1_LG
dir_0.2_LG
dir_0.3_LG
dir_0.4_LG
dir_0.5_LG
dir_0.6_LG
dir_0.7_LG
dir_0.8_LG
dir_0.9_LG
dir_1_LG
dir_0.1_C
dir_0.2_C
dir_0.3_C
dir_0.4_C
dir_0.5_C
dir_0.6_C
dir_0.7_C
dir_0.8_C
dir_0.9_C
dir_1_C
di

,x_0.1_LT,x_0_LT,delta_x_0.1_LT,playDirection
0,41.28,41.38,-0.10,right
1,109.62,109.62,-0.00,left
2,78.30,78.27,-0.03,left
3,50.63,50.62,-0.01,left
4,55.81,55.78,-0.03,left
...,...,...,...,...
8066,19.51,19.46,0.05,NaN
8067,33.32,33.33,-0.01,NaN
8068,36.18,36.19,-0.01,NaN
8069,28.09,28.12,-0.03,NaN


In [89]:
# Principal Component Analysis
cols_of_interest = ['action_time'] + delta_cols
data_all = data_transformed.dropna()
data = data_all.select_dtypes(['number'])
data = data[cols_of_interest]
x = data.loc[:, data.columns != 'action_time']
y = data['action_time']
pca = PCA(n_components = 35) # in theory, one per variable per position
print("pca fitting")
principal_components = pca.fit_transform(x)
print(principal_components)
principal_components_df = pd.DataFrame(principal_components)
principal_components_df.to_csv("35_principal_components_time_to_throw.csv")

pca fitting
[[  2.48053252   2.30231339   1.38547418 ...   0.56454111  13.43470191
    6.08916357]
 [349.95233954  43.90541681 -34.89196529 ...  -5.97867261 -25.9317159
  -51.37017636]
 [  5.8281259   -1.79242845   2.69609504 ...  -1.16649887  -0.78021608
    3.15763679]
 ...
 [ -6.83382256   1.41242739 -11.11658158 ...  -4.52044     -3.28225448
    4.51533941]
 [  7.74950203   0.76262971 -11.91659775 ...   5.98594597  -1.06322629
    4.13080111]
 [-14.11137385 -23.24213394  -8.43440329 ...   1.10356031  -2.36437499
    8.21776578]]


In [90]:
# Prep Train and Test Data
label = data['action_time']
principal_components_df['action_time'] = label
X = data.loc[:, data.columns != 'action_time']
y = data['action_time']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print("completed train-test split")

completed train-test split


In [91]:
print(datetime.now())
random_grid = { # FLAG
    'bootstrap': [True, False],
    'max_depth': [5, 10, 20, 50],
    'max_features': ['sqrt', 'log2'],
    'min_samples_leaf': [3, 5, 10],
    'min_samples_split': [3, 5, 15],
    'n_estimators': [250, 500, 1000, 2000],
}

rf_tuning = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf_tuning, param_distributions = random_grid, n_iter = 30, cv = 3, verbose = 2, random_state = 42, n_jobs = -1)
rf_fit_output = rf_random.fit(x_train, y_train)
print(datetime.now())

2022-12-07 23:19:25.512967
Fitting 3 folds for each of 30 candidates, totalling 90 fits
2022-12-07 23:21:02.077021


In [92]:
rf_tuned = rf_fit_output.best_estimator_
rf_tuned.fit(x_train, y_train)

RandomForestRegressor(bootstrap=False, max_depth=50, max_features='sqrt',
                      min_samples_leaf=10, min_samples_split=5,
                      n_estimators=1000)

In [93]:
# Use the forest's predict method on the test data
predictions = rf_tuned.predict(x_test)
# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')
print(mean_squared_error(y_test, predictions))
print(rf_tuned.score(x_test, y_test))

Mean Absolute Error: 0.83 degrees.
Accuracy: 71.31 %.
1.7654764151944013
0.05685678030867902


In [ ]:
test_set_with_predictions = pd.DataFrame(columns = {'actual': y_test, 'predicted': predictions})
test_set_with_predictions['actual'] = y_test
test_set_with_predictions['predicted'] = predictions
test_set_with_predictions.to_csv("test_set_with_predictions.csv")
#for i in range(len(y_test)):
#    print(y_test[i], predictions[i])